In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
import pickle
import joblib
import warnings
warnings.filterwarnings("ignore")

In [15]:
df = pd.read_csv('C:\\AIMANA\\AcoTubos\\data\\acotubo_poc_16_06_2025.csv')
df = df.dropna(subset= ['Desconto', 'Margem'])
df.head()

,Year,day_of_year,FaixaPeso,ProdutoFamilia,ProdutoDescricao,ProdutoGrupoSOP,Canal,EmpresaNome,ClienteCNPJCPF,totalSold,TotalQuoted,nuPrecoGerenciaTotal,Desconto,ConversionRate,Margem
0,2022,3,1.001 A 2.000 KG,AÇOS ESPECIAIS,"ACO TREF RED 11SMN37 25,40 MM CNA N",04-AÇO RESSULFURADO,VAREJO,TUBOS E AÇOS PR,06271863000180,0.180000,0.180000,2561.40000,0.000000,1.000000,20.031008
1,2022,3,1.001 A 2.000 KG,AÇOS LONGOS,"ACO LAM RED SAE 1045 FX 1.3/4"" CNA N",05-AÇO FAIXA,VAREJO,TUBOS E AÇOS MTZ,54183819000160,1.517040,1.517037,20282.78469,-6.503497,1.000002,32.440000
2,2022,3,1.001 A 2.000 KG,AÇOS LONGOS,"ACO TREF RED SAE 1045 2.1/2"" CNA N",06-AÇO TREFILADO,VAREJO,TUBOS E AÇOS MG,17732545000276,0.750000,0.750000,11587.50000,0.000000,1.000000,24.947977
3,2022,3,1.001 A 2.000 KG,AÇOS LONGOS,ACO TREF RED SAE 1045 20 MM CNA N,06-AÇO TREFILADO,VAREJO,TUBOS E AÇOS PR,06271863000180,1.577280,1.577280,20914.73280,0.000000,1.000000,24.855769
4,2022,3,1.001 A 2.000 KG,AÇOS LONGOS,"AÇO LAM RED SAE 1045 FX 1.1/2"" CNA N",05-AÇO FAIXA,VAREJO,TUBOS E AÇOS MG,17732545000276,0.429734,0.429734,5603.73136,0.000000,1.000000,24.945205


In [16]:
def predict_preco(df, 
                     limite: float,
                     conversion_model_path,
                     margem_model_path,
                     ano: int,
                     dia_ano: int, 
                     faixa_peso: str,
                     produto_familia: str, 
                     produto_descricao: str,
                     produto_grupo_sop: str,
                     canal: str,
                     empresa_nome: str,
                     cliente_cnpj_cpf: str,
                     nuPrecoGerenciaTotal: float):
    
    conversion_model = joblib.load(conversion_model_path)
    margem_model = joblib.load(margem_model_path)
    
    #descontos
    range_descontos = np.arange(1, 20.5, 0.2)

    #inputs
    user_inputs = {
        'Year': ano,
        'day_of_year': dia_ano,
        'FaixaPeso': faixa_peso,
        'ProdutoFamilia': produto_familia,
        'ProdutoDescricao': produto_descricao,
        'ProdutoGrupoSOP': produto_grupo_sop,
        'Canal': canal,
        'EmpresaNome': empresa_nome,
        'ClienteCNPJCPF': cliente_cnpj_cpf, 
        'nuPrecoGerenciaTotal': nuPrecoGerenciaTotal
    }

    user_inputs2 = {
        'totalSold': df[df['ProdutoDescricao'] == user_inputs['ProdutoDescricao']]['totalSold'].iloc[-1],
        'TotalQuoted': df[df['ProdutoDescricao'] == user_inputs['ProdutoDescricao']]['TotalQuoted'].iloc[-1]  
    }


    df_inputs = pd.DataFrame([{
        **user_inputs,
        **user_inputs2,
        'Desconto': desconto
    } for desconto in range_descontos])


    conversion_predictions = conversion_model.predict(df_inputs)
    margem_predictions = margem_model.predict(df_inputs)


    #predictions dataframe
    df_results = pd.DataFrame({
    'Desconto' : range_descontos,
    'ConversionRate_Predicted': conversion_predictions,
    'Margem_Predicted': margem_predictions
    })

    #extraindo somente os registros que possuem um conversion rate acima do limite calculado em relação ao ultimo registro do produto
    limit = df[df['ProdutoDescricao'] == user_inputs['ProdutoDescricao']].iloc[-1]['ConversionRate'] * limite

    #extraindo o valor de máximo de margem encontrado
    max_margem = df_results[df_results['ConversionRate_Predicted'] >= limit]['Margem_Predicted'].max()

    #descontos encontrados 
    desconto_encontrado = df_results[df_results['Margem_Predicted'] == max_margem]['Desconto'].tolist()

    #conversion rate encontrado (dentro da margem maxima)
    conversion_rate_encontrado = df_results[df_results['Margem_Predicted'] == max_margem]['ConversionRate_Predicted'].values[0]


    range_preco= []
    for desconto in desconto_encontrado:
        if desconto < 0:
            preco= nuPrecoGerenciaTotal - nuPrecoGerenciaTotal*(desconto/100)
            range_preco.append(preco)
        else:
            preco= nuPrecoGerenciaTotal - nuPrecoGerenciaTotal*(desconto/100)
            range_preco.append(preco)
    
    
    #resultados
    resultados= {
            "Margem Máxima encontrada": max_margem, 
            "Conversion Rate encontrado": conversion_rate_encontrado, 
            "Desconto(s) encontrado(s)": desconto_encontrado,
            "Preço Original": nuPrecoGerenciaTotal,
            "Preço(s) encontrado(s)": range_preco
            }
    
    
    #printando resultados
    for chave, valor in resultados.items():
        print(f'{chave}: {valor}\n') 

In [17]:
predict_preco(df,
                0.9,
                'C:\\AIMANA\\AcoTubos\\app\\models\\2_poc_conversion_model_rf_df_16062025.joblib',
                'C:\\AIMANA\\AcoTubos\\app\\models\\2_poc_margem_model_xgb_df_16062025.joblib',
                2022,
                5,
                '501 A 1.000 KG',
                'AÇOS ESPECIAIS',
                'AÇO LAM RED SAE 1045 QUAL 5" CNA N',
                '03-AÇO QUALIDADE',
                'VAREJO',
                'TUBOS E AÇOS MTZ',
                '00246591000181',
                1500
                )

Margem Máxima encontrada: 19.487194061279297

Conversion Rate encontrado: 99.78186501667153

Desconto(s) encontrado(s): [1.2, 1.4]

Preço Original: 1500

Preço(s) encontrado(s): [1482.0, 1479.0]



In [18]:
input={
  "limite": 0.9,
  "Year": 2022,
  "day_of_year": 5,
  "FaixaPeso": "501 A 1.000 KG",
  "ProdutoFamilia": "AÇOS ESPECIAIS",
  "ProdutoDescricao": "AÇO LAM RED SAE 1045 QUAL 5" CNA N",
  "ProdutoGrupoSOP": "03-AÇO QUALIDADE",
  "Canal": "VAREJO",
  "EmpresaNome": "TUBOS E AÇOS MTZ",
  "ClienteCNPJCPF": "00246591000181",
  "nuPrecoGerenciaTotal": 1500
}

SyntaxError: unterminated string literal (detected at line 7) (3199439178.py, line 7)

In [63]:
texto = """a b c d " e a f """
texto

'a b c d " e a f '

In [64]:
texto.replace('"', "'")


"a b c d ' e a f "

In [65]:
a= "a b c d ' e a f "
a

"a b c d ' e a f "